# 1. 2023年10月6日开始重构

- 1.akshare有一个接口更新了名字
- 2.券商研报的表头更新了
- 3.通过萝卜投研爬取10年pe平均值做了线程池优化（12个workers让本来50分钟的爬取减少到5分钟）

# 2024年7月3日更新券商研报的表头，其他都没变化
# 2024年8月19日更新akshare=1.14.62 接口行业的表头

In [1]:
import akshare as ak
# (base)
ak.stock_profit_forecast_em().head(5)

  0%|          | 0/6 [00:00<?, ?it/s]

d:\miniforge3\envs\python310forclawer\lib\site-packages\akshare\stock_fundamental\stock_profit_forecast_em.py:115: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  big_df["机构投资评级(近六个月)-买入"].fillna(0, inplace=True)
d:\miniforge3\envs\python310forclawer\lib\site-packages\akshare\stock_fundamental\stock_profit_forecast_em.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work 

,序号,代码,名称,研报数,机构投资评级(近六个月)-买入,机构投资评级(近六个月)-增持,机构投资评级(近六个月)-中性,机构投资评级(近六个月)-减持,机构投资评级(近六个月)-卖出,2023预测每股收益,2024预测每股收益,2025预测每股收益,2026预测每股收益
0,1,600519,贵州茅台,57,47.0,10.0,0.0,0,0,59.492280,69.271649,79.394947,90.274107
1,2,603605,珀莱雅,49,38.0,11.0,0.0,0,0,3.009063,3.853122,4.748061,5.756208
2,3,300750,宁德时代,48,37.0,11.0,0.0,0,0,10.030275,11.376106,13.566106,15.844267
3,4,600600,青岛啤酒,48,36.0,11.0,1.0,0,0,3.128472,3.688854,4.217771,4.732745
4,5,000858,五粮液,47,39.0,8.0,0.0,0,0,7.783008,8.740617,9.788426,10.950023


In [1]:
#!/usr/bin/env python=3.10
# coding=utf-8
# author: KarlQu
# Version: 1.1
# Date: 2023-10-6
# webdriver=Firefox2023.04
# pip install akshare -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host=mirrors.aliyun.com  --upgrade
#  mamba install -c conda-forge selenium=4.13.0
# selenium 4 与 selenium 3 对对象选择语法做了修改
#  mamba install -c conda-forge jupyter
# akshare.__version__ = '1.11.23 '
# !最重要的是g和g的可达性
# akshare更新非常频繁
# pip install akshare --upgrade -i https://pypi.org/simple

# 是否需要更新10年pe均值和券商研报, pe_ave_10(ifupdate = False), 输出位置 f'./backups/{DATE}/nets_end.xlsx'

import numpy as np
import pandas as pd
import requests
import akshare as ak
import tqdm
import time
import datetime as dt
import os
import calendar
from selenium import webdriver
# from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
from concurrent import futures

DATE = dt.date.today() # str(DATE) ; dt.date.today().year

UA = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0'
}

def open_file(file_name:str, upperdir = 'backups'):
    """创建文件夹
    Parameters
    ----------
    file_name : 日期字符串
    upperdir : 上级目录名
    """
    if os.path.exists(f'./{upperdir}/' + file_name):
        pass
    else:
        os.mkdir(f'./{upperdir}/' + file_name)

def update_g(ifupdate = False) -> pd.DataFrame:
    """akshare的个股研报接口

    Parameters
    ----------
    ifupdate : bool, optional
        _description_, by default False

    Returns
    -------
    pd.DataFrame
        columns = ['代码','名称','研报数','2021预测每股收益','2022预测每股收益','2023预测每股收益','2024预测每股收益']
        
        共4年的每股收益,2023年更新为'2022预测每股收益','2023预测每股收益','2024预测每股收益','2025预测每股收益'

    """
    if ifupdate:
        open_file(str(DATE), upperdir = 'backups')
        profit_forecast = ak.stock_profit_forecast_em().iloc[:,[1,2,3,9,10,11,12]].drop_duplicates()
        profit_forecast.to_excel(f'./backups/{DATE}/profit_forecast.xlsx',index=False)

    else:
        latest_file = list(os.walk('./backups/'))[0][1][-1]
        profit_forecast = pd.read_excel(f'./backups/{latest_file}/profit_forecast.xlsx',dtype={'代码':'str'})
    # 调整g,收益>0、每期g>10%、每期g可信值35%。 得到调整的几何均数g
    profit_forecast = profit_forecast[(profit_forecast['2023预测每股收益']>0) & (profit_forecast['2024预测每股收益']>0 )& ( profit_forecast['2025预测每股收益']>0)]
    profit_forecast
    profit_forecast1 = profit_forecast.iloc[:,[3,4,5,6]].pct_change(axis=1)[['2024预测每股收益','2025预测每股收益',	'2026预测每股收益']]
    profit_forecast1
    profit_forecast2 = profit_forecast1[(profit_forecast1['2024预测每股收益']>0.1) & (profit_forecast1['2025预测每股收益']>0.1 )& ( profit_forecast1['2026预测每股收益']>0.1)]
    profit_forecast2
    profit_forecast2[profit_forecast2>0.35] = 0.35
    profit_forecast3 = profit_forecast2 + 1
    g_scaled = ((profit_forecast3['2024预测每股收益']*profit_forecast3['2025预测每股收益']*profit_forecast3['2026预测每股收益'])**(1/3)-1)
    g_scaled.name = 'g'
    g = pd.concat([g_scaled,profit_forecast],axis=1,join='inner')[['代码','名称','g','研报数']].reset_index(drop=True)
    return g


def ten_year_ago(date: dt.date = DATE) -> str:
        """10年前日期，月底对应月底
        Parameters
        ----------
        date : datetime.date
        Returns
        -------
        str
        """
        year = date.year - 10
        day = min(date.day, calendar.monthrange(year, date.month)[1])
        dt = date.replace(year=year, month=date.month, day=day)
        # print(dt)
        bigen_date = str(dt).replace('-','')
        return bigen_date

def pe_ave_10(date:str = str(DATE), ifupdate = False):
    """获取or更新10年pe均值和券商研报的预期每股收益

    Parameters
    ----------
    date : str, optional
        _description_, by default str(DATE)
    ifupdate : bool, optional
        _description_, by default False
    """
    if ifupdate:
        g = update_g(ifupdate=ifupdate)
        stock_ids = g.代码.to_list() # 每次爬pe历史均值的时候都更新一下券商研报
        print('代码前五个')
        print(stock_ids[0:5])
        # 1. selenium登录获取cookies
        denglu_link = 'https://robo.datayes.com/v2/landing/peband'
        cruser = webdriver.Firefox()
        cruser.get(denglu_link)
        input("扫码登录后按回车键")
        # WebDriverWait(cruser,60).until(EC.presence_of_all_elements_located((By.ID, 'app')))
        cookies = {x['name']:x['value'] for x in cruser.get_cookies()}
        print(cookies)
        end_date = date.replace('-','')
        begin_date = ten_year_ago()
        ave_pettm = []
        executor = futures.ThreadPoolExecutor(max_workers=12)
        fs = []
        total_tasks = len(stock_ids)
        with tqdm.tqdm(total=total_tasks) as pbar:
            for ids in stock_ids:
                def get_ids_peave(ids, begin_date, end_date, cookies, UA, ave_pettm):
                    try:
                        url = f'https://gw.datayes.com/rrp_adventure/web/stockModel/band/{ids}' # 2022old
                        response = requests.get(url=url,
                                                params= {
                                                    'apiType': '4',
                                                    'category': '1',    # 1为PE，2为PB
                                                    'subCategory': '1',
                                                    'flag': '-1',
                                                    'beginDate': f'{begin_date}',
                                                    'endDate': f'{end_date}'
                                                },
                                                cookies=cookies,
                                                headers=UA)
                        ave_pettm.append([ids, response.json()['data']['mean']])
                        pbar.update(1) # 更新tqdm进度条用
                        time.sleep(np.random.rand()) # 1536/1536 [33:05<00:00,  1.29s/it]
                        # print(f'正常{response.status_code} , code: {ids}')
                    except:
                        print(f'异常{response.status_code} , code: {ids}')
                        pass

                f= executor.submit(get_ids_peave, ids, begin_date, end_date, cookies, UA, ave_pettm)
                fs.append(f)
            futures.wait(fs)
        cruser.quit()
        # 3. 本地保存
        pe_ave = pd.DataFrame(ave_pettm, columns=['code','ave_pettm'])
        open_file(str(DATE), upperdir = 'backups')
        pe_ave.to_excel(f'./backups/{DATE}/avepettm_backup.xlsx',index=False)
        latest_file = DATE
    else:
        latest_file = list(os.walk('./backups/'))[0][1][-1]
        print(f'研报和10年期pe平均值的最近一次更新时间为 {latest_file}')
        g = update_g(ifupdate=ifupdate)
        pe_ave = pd.read_excel(f'./backups/{latest_file}/avepettm_backup.xlsx',dtype={'code':'str'})
    nets = pd.merge(g.rename(columns={'代码':'code'}) , pe_ave , on='code',how='left')
    nets['peg'] = (nets.ave_pettm / (100*nets.g))
    nets1 = nets[(nets['peg']<1.2)]
    # (nets['peg']<1.2).sum()
    # 2. 得到筛选后的股票代码，后续可以爬取pettm和dvttm了
    stock_ids_selected = nets1.code.to_list()
    print('DEBUG:print 代码前五')
    print(stock_ids_selected[0:5])
    pe_ttm = []
    for ids in tqdm.tqdm(stock_ids_selected):
        try:
            rrr = ak.stock_a_indicator_lg(symbol=f'{ids}').iloc[-1,[0,2,7]] # pettm and dvttm 接口
            # print(f"rrr {rrr}")
            rrr['industry'] = ak.stock_individual_info_em(symbol=f"{ids}").loc[6].value # 行业接口
            # print(f"rrr with industry {rrr}")
            rrr.name = ids
            pe_ttm.append(rrr)
        except:
            print(f'{ids} 查无!')
        continue

    pe_and_dv = pd.concat(pe_ttm,axis=1).T
    pe_and_dv_backup = pe_and_dv.reset_index(drop=False)
    pe_and_dv_backup = pe_and_dv_backup.rename(columns={'index':'code'}) 
    open_file(str(DATE), upperdir = f'backups/{latest_file}')
    pe_and_dv_backup.to_excel(f'./backups/{latest_file}/{DATE}/pe_and_dv.xlsx',index=False)
    # pe_ttm = pd.read_excel(f'./backups/{latest_file}/{DATE}/pe_and_dv.xlsx',dtype={'code':'str'})
    nets2 = pd.merge(nets1,pe_and_dv_backup,on='code',how='left')
    nets3 = nets2[nets2.pe_ttm<nets2.ave_pettm]
    nets3['peg'] = (nets3.pe_ttm / (100*nets3.g))
    nets3['exp_reward_yearly'] = 100*((nets3.ave_pettm/nets3.pe_ttm)**(1/3)*(1+nets3.g)-1)
    nets_end = nets3[nets3.exp_reward_yearly>35].drop_duplicates()
    nets_end.to_excel(f'./backups/{latest_file}/{DATE}/nets_end.xlsx',index=False)   
    print('done! open nets_end.xlsx')

## 行业接口的表头由[2]到[6]

In [2]:
ak.stock_individual_info_em(symbol=f"603198").loc[6].value

'酿酒行业'

## legu个股数据容易错
> 1.14.61的错误是表格里面数据有null
> 
> 1.14.62把null都更新为None了就好了

In [3]:
from datetime import datetime
from hashlib import md5

import pandas as pd
import requests
from bs4 import BeautifulSoup

from akshare.utils.cons import headers

def get_cookie_csrf(url: str = "") -> dict:
    """
    乐咕乐股-主板市盈率
    https://legulegu.com/stockdata/shanghaiPE
    :return: 指定市场的市盈率数据
    :rtype: pandas.DataFrame
    """
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, features="lxml")
    csrf_tag = soup.find(name="meta", attrs={"name": "_csrf"})
    csrf_token = csrf_tag.attrs["content"]
    headers.update({"X-CSRF-Token": csrf_token})
    return {"cookies": r.cookies, "headers": headers}


def get_token_lg() -> str:
    """
    生成乐咕的 token
    https://legulegu.com/s/002488
    :return: token
    :rtype: str
    """
    current_date_str = datetime.now().date().isoformat()
    obj = md5()
    obj.update(current_date_str.encode("utf-8"))
    token = obj.hexdigest()
    return token

symbol = "603198"
url = "https://legulegu.com/api/s/base-info/"
token = get_token_lg()
params = {"token": token, "id": symbol}
r = requests.post(
    url,
    params=params,
    **get_cookie_csrf(url="https://legulegu.com/"),
)
temp_json = r.json()
temp_df = pd.DataFrame(
    temp_json["data"]["items"],
    columns=temp_json["data"]["fields"],
)
print(temp_df)
temp_df["trade_date"] = pd.to_datetime(temp_df["trade_date"]).dt.date
temp_df[temp_df.columns[1:]] = temp_df[temp_df.columns[1:]].astype(float)
temp_df.sort_values(by=["trade_date"], inplace=True, ignore_index=True)
if len(set(temp_df["trade_date"])) <= 0:
    raise ValueError("数据获取失败, 请检查是否输入正确的股票代码")

     trade_date       pe   pe_ttm      pb      ps  ps_ttm dv_ratio  dv_ttm  \
0      20240819  18.1064  15.9164  4.7765  6.1642  5.6264   2.1244  2.5106   
1      20240816  17.7357   16.225  4.9426   6.038  5.6919   2.1688  2.5631   
2      20240815  17.5749  16.0779  4.8978  5.9833  5.6403   2.1886  2.5865   
3      20240814  17.5189  16.0267  4.8822  5.9642  5.6223   2.1956  2.5948   
4      20240813  17.9385  16.4106  4.9991  6.1071   5.757   2.1442  2.5341   
...         ...      ...      ...     ...     ...     ...      ...     ...   
2241   20150603  40.9079  40.9079  5.4597  6.7267  6.7267     None    None   
2242   20150602   37.192   37.192  4.9638  6.1157  6.1157     None    None   
2243   20150601  33.8049  33.8049  4.5117  5.5587  5.5587     None    None   
2244   20150529  30.7302  30.7302  4.1014  5.0531  5.0531     None    None   
2245   20150528  27.9351  27.9351  3.7283  4.5935  4.5935     None    None   

       total_mv  
0     4142400.0  
1     4057600.0  
2     402

In [4]:
# pe_ave_10(ifupdate=True)
pe_ave_10(ifupdate=False)

研报和10年期pe平均值的最近一次更新时间为 2024-08-18


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11028\1786775755.py:79: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  profit_forecast1 = profit_forecast.iloc[:,[3,4,5,6]].pct_change(axis=1)[['2024预测每股收益','2025预测每股收益',	'2026预测每股收益']]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11028\1786775755.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit_forecast2[profit_forecast2>0.35] = 0.35
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11028\1786775755.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

DEBUG:print 代码前五
['600426', '603198', '601633', '002648', '002078']


 51%|█████     | 200/392 [06:32<01:49,  1.75it/s]

000059 查无!


...

 99%|█████████▉| 389/392 [10:52<00:01,  2.98it/s]

600469 查无!


100%|██████████| 392/392 [10:56<00:00,  1.67s/it]

done! open nets_end.xlsx



C:\Users\Administrator\AppData\Local\Temp\ipykernel_11028\1786775755.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nets3['peg'] = (nets3.pe_ttm / (100*nets3.g))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11028\1786775755.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nets3['exp_reward_yearly'] = 100*((nets3.ave_pettm/nets3.pe_ttm)**(1/3)*(1+nets3.g)-1)


# 后记
## 1.end 使用结果来选股
> 2022-10-14选的股，到现在(2024.7.3)看都有了35%的收益

**net_end.xlsx**

- 1 首先用peg和pe_ttm做降序排序
- 2 看pe_ttm与10年平均pe的距离
- 3 优先选dv_ttm（股息率）高于3.4的
- 4 对比g（券商研报）和pe_ttm看能不能支撑pe的回报率

# 2. AkShare的legulegu.com/robots.txt协议对爬虫没做限制,但是！？
尝试也改造成线程池加速
> 被封ip了。。。

In [3]:
import os
list(os.walk('./backups/')) # [0][1][-1]

[('./backups/', ['2022-10-16', '2023-10-06'], []),
 ('./backups/2022-10-16',
  [],
  ['avepettm_backup.xlsx',
   'nets_end.xlsx',
   'pe_and_dv.xlsx',
   'profit_forecast.xlsx']),
 ('./backups/2023-10-06',
  [],
  ['avepettm_backup.xlsx',
   'nets_end.xlsx',
   'pe_and_dv.xlsx',
   'profit_forecast.xlsx'])]